In [ ]:
import numpy as np
import pandas as pd
import nd2reader
import matplotlib.pyplot as plt
import holoviews as hv
from holoviews.operation.datashader import regrid
import skimage.filters
import skimage.feature
import scipy.stats
import scipy.ndimage
import peakutils
from tqdm import tnrange, tqdm_notebook
import dask
import dask.array as da
import distributed
from distributed import Client, LocalCluster, progress
from dask_jobqueue import SLURMCluster
from cytoolz import partial, compose, juxt
from itertools import repeat
from glob import glob
import cachetools
import numpy_indexed
import pickle
import pyarrow as pa
import warnings
import os
import re
from numbers import Integral
from dask.delayed import Delayed

In [ ]:
%load_ext autoreload
%autoreload 2

In [ ]:
# from segmentation import *
# from util import *
# from matriarch_stub import *
import segmentation
import matriarch_stub

In [ ]:
plt.rcParams["figure.figsize"] = (12, 12)
hv.extension("bokeh")

# Run

In [ ]:
dask.config.config["distributed"]["scheduler"]["allowed-failures"] = 20
# dask.config.config['distributed']['worker']['memory'] = {'target': 0.4,
#                                                         'spill': 0.5,
#                                                         'pause': 0.9,
#                                                         'terminate': 0.95}

In [ ]:
cluster = SLURMCluster(
    queue="short",
    walltime="02:00:00",
    memory="8GB",
    local_directory="/tmp",
    log_directory="/home/jqs1/projects/molecule-counting/log",
    cores=1,
    processes=1,
)
# diagnostics_port=('127.0.0.1', 8787),
# env_extra=['export PYTHONPATH=\"/home/jqs1/projects/matriarch\"'])
client = Client(cluster)

In [ ]:
cluster

In [ ]:
cluster.scale(40)

# ND2

In [ ]:
base_filename = "/n/scratch2/jqs1"

In [ ]:
nd2 = nd2reader.ND2Reader(
    os.path.join(base_filename, "191010/tmp/191010_noise_surpression_control.nd2")
)

In [ ]:
num_ts = nd2._parser._raw_metadata.image_metadata[b"SLxExperiment"][b"uLoopPars"][
    b"uiCount"
]

In [ ]:
positions = nd2._parser._raw_metadata.image_metadata[b"SLxExperiment"][
    b"ppNextLevelEx"
][b""][b"uLoopPars"][b"Points"][b""]

In [ ]:
num_positions = len(positions)

In [ ]:
plt.scatter(
    *np.array([(d[b"dPosX"], d[b"dPosY"]) for d in positions]).T,
    c=range(len(positions)),
)

In [ ]:
plt.imshow(nd2.get_frame_2D(v=40, c=1, t=30))

In [ ]:
plt.imshow(nd2.get_frame_2D(v=40, c=1, t=20))

In [ ]:
nd2.metadata

In [ ]:
@dask.delayed
def func(filename):
    img = nd2reader.ND2Reader(filename)
    labels = segmentation.segment(img)
    return np.array(matriarch_stub.map_over_labels(np.mean, labels, img))


futures = client.compute(
    [
        [
            func(filename)
            for filename in sorted(glob(filename_t1.replace("t000001", "t*")))
        ]
        for filename_t1 in sorted(filenames)
    ]
)

# TIFF

In [ ]:
base_filename = "/n/scratch2/jqs1"
filenames = glob(os.path.join(base_filename, "191010/nsc*/xy*c1t000010.tif"))

In [ ]:
f = skimage.external.tifffile.TiffFile(filenames[0])

In [ ]:
f.pages[0].tags

In [ ]:
f.pages[0].tags["40961"].value

In [ ]:
with skimage.external.tifffile.TiffFile(filenames[0]) as f:
    f.pages[0].tags

In [ ]:
res2 = dict(zip(filenames, res))

In [ ]:
@dask.delayed
def func(filename):
    img = skimage.io.imread(filename)
    labels = segmentation.segment(img)
    return np.array(matriarch_stub.map_over_labels(np.mean, labels, img))


futures = client.compute(
    {
        filename_t1: [
            func(filename)
            for filename in sorted(glob(filename_t1.replace("t000010", "t*")))
        ]
        for filename_t1 in sorted(filenames)
    }
)

In [ ]:
res = client.gather(futures)

In [ ]:
import pickle

with open("nsc_wt.pkl", "wb") as f:
    pickle.dump(res, f)

In [ ]:
%store res

In [ ]:
res.keys()

In [ ]:
CLOSEDLOOP = [
    "/n/scratch2/jqs1/191010/nsc_closedloop/xy085c1t000010.tif",
    "/n/scratch2/jqs1/191010/nsc_closedloop/xy090c1t000010.tif",
    "/n/scratch2/jqs1/191010/nsc_closedloop/xy110c1t000010.tif",
    "/n/scratch2/jqs1/191010/nsc_closedloop/xy083c1t000010.tif",
    "/n/scratch2/jqs1/191010/nsc_closedloop/xy086c1t000010.tif",
    "/n/scratch2/jqs1/191010/nsc_closedloop/xy105c1t000010.tif",
]
OPENLOOP = [
    "/n/scratch2/jqs1/191010/nsc_openloop/xy022c1t000010.tif",
    "/n/scratch2/jqs1/191010/nsc_openloop/xy034c1t000010.tif",
    "/n/scratch2/jqs1/191010/nsc_openloop/xy043c1t000010.tif",
    "/n/scratch2/jqs1/191010/nsc_openloop/xy075c1t000010.tif",
    "/n/scratch2/jqs1/191010/nsc_openloop/xy023c1t000010.tif",
    "/n/scratch2/jqs1/191010/nsc_openloop/xy033c1t000010.tif",
]
ALL = CLOSEDLOOP + OPENLOOP

In [ ]:
for f in CLOSEDLOOP:
    plt.hist(res[f][1], log=True, bins=20)
for f in OPENLOOP:
    plt.hist(res[f][1], log=True, bins=20)

In [ ]:
for f in ALL:
    plt.plot([np.mean(dd[1:]) for dd in res[f]])

In [ ]:
for f in ALL:
    plt.plot([np.std(dd[1:]) for dd in res[f]])

In [ ]:
for f in CLOSEDLOOP:
    plt.plot([scipy.stats.variation(dd[1:]) for dd in res[f]])
for f in OPENLOOP:
    plt.plot([scipy.stats.variation(dd[1:]) for dd in res[f]], linestyle=":")